# Packages

In [179]:
# Data wrangling
import pandas as pd
import polars as pl
import polars.selectors as cs
import numpy as np

# Visualisation
import plotnine as pn
import matplotlib.pyplot as plt
from mizani.formatters import comma_format, custom_format, currency_format, percent_format
from IPython.display import clear_output, display
import matplotlib.font_manager as fm
import matplotlib as mpl
from matplotlib import rc
import plotly.express as px

# Utils
import os
from tqdm.notebook import tqdm
import itertools
import yaml
import warnings
import time
import holidays
import pickle
import datetime

# Modelling
from sklearn.linear_model import Lasso
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import (
    StandardScaler,
    OneHotEncoder,
    FunctionTransformer,
)
from sklearn.metrics import (
    r2_score,
    mean_absolute_error,
    mean_absolute_percentage_error,
    root_mean_squared_error,
)
from sklearn.pipeline import FeatureUnion, make_pipeline, Pipeline
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer, KNNImputer
from sklearn.feature_selection import VarianceThreshold

import ray
from ray import train, tune
from ray.tune.search.optuna import OptunaSearch
from ray.tune.schedulers import ASHAScheduler


import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.metrics import RootMeanSquaredError

rc('text', usetex=False)

jama_colour = [
    "#374e55",
    "#df8f44",
    "#00a1d5",
    "#b24745",
    "#79af97",
    "#6a6599",
    "#80796b",
]

pd.set_option("display.max.columns", 500)
pd.set_option("display.max.columns", 500)


theme_academic = pn.theme(
    text=pn.element_text(family="Latin Modern Roman"),
    plot_title=pn.element_text(weight="bold", size=14, ha="center"),
    legend_text=pn.element_text(size=9),  # Smaller font for legend items
    panel_background=pn.element_rect(fill="white"),  # Clean white background
    panel_border=pn.element_rect(color="grey", size=0.5),
    axis_ticks=pn.element_line(color="grey"),
    panel_grid_major=pn.element_line(color="grey", size=0.1, alpha=0.3),
    panel_grid_minor=pn.element_line(color="grey", size=0.1, alpha=0.3),
    legend_background=pn.element_rect(fill="white", color=None),
    legend_key=pn.element_rect(fill="white", color=None),
    plot_margin=0.02,
    figure_size=(6, 4),  # Set default figure size (width, height in inches)
)

%matplotlib inline

# Loading the data

In [129]:
df = pl.read_csv(
    "../0_data/preprocessed/df_final_reduced.csv", try_parse_dates=True
).filter(pl.col("datetime") >= pd.Timestamp("2021-09-01 00:00"))

df.head()

datetime,kWh,Zurich_shortwave_radiation,Zurich_soil_temperature_7_to_28cm
datetime[μs],f64,f64,f64
2021-09-01 00:00:00,5.1633e6,0.0,16.042
2021-09-01 01:00:00,5.0777e6,0.0,15.892
2021-09-01 02:00:00,4.9315e6,0.0,16.042
2021-09-01 03:00:00,4.7878e6,0.0,15.942
2021-09-01 04:00:00,4.7038e6,0.0,15.792001


In [130]:
df.shape

(26304, 4)

# Missing values

Just forward fill for now.

In [131]:
df = df.fill_null(strategy="forward")

# Calendar Features

In [132]:
df = df.with_columns(
    day_of_month=pl.col("datetime").dt.day(),
    day_of_year=pl.col("datetime").dt.ordinal_day(),
    day_of_week=pl.col("datetime").dt.weekday(),
    month=pl.col("datetime").dt.month(),
    hour=pl.col("datetime").dt.hour(),
    year=pl.col("datetime").dt.year(),
)

# Holidays

In [133]:
# Define the region (Canton of Berne) and the country (Switzerland)
country = "CH"
prov = "ZH"

# Create a list of the regional holidays for the canton of Berne
regional_holidays = holidays.CH(
    years=df["datetime"].dt.year().unique().to_list(), prov=prov
)

In [134]:
holiday_df = pl.DataFrame(
    {
        "holiday_name": list(regional_holidays.values()),
        "holiday_date": list(regional_holidays.keys()),
    }
).sort("holiday_date")

holiday_df.head()

holiday_name,holiday_date
str,date
"""Neujahrestag""",2021-01-01
"""Berchtoldstag""",2021-01-02
"""Karfreitag""",2021-04-02
"""Ostermontag""",2021-04-05
"""Tag der Arbeit""",2021-05-01


In [135]:
df = (
    df.with_columns(date=pl.col("datetime").dt.date())
    .join(holiday_df, how="left", left_on="date", right_on="holiday_date")
    .drop("date")
    .with_columns(holiday_name=pl.col("holiday_name").fill_null("no_holiday"))
)

df.head()

datetime,kWh,Zurich_shortwave_radiation,Zurich_soil_temperature_7_to_28cm,day_of_month,day_of_year,day_of_week,month,hour,year,holiday_name
datetime[μs],f64,f64,f64,i8,i16,i8,i8,i8,i32,str
2021-09-01 00:00:00,5.1633e6,0.0,16.042,1,244,3,9,0,2021,"""no_holiday"""
2021-09-01 01:00:00,5.0777e6,0.0,15.892,1,244,3,9,1,2021,"""no_holiday"""
2021-09-01 02:00:00,4.9315e6,0.0,16.042,1,244,3,9,2,2021,"""no_holiday"""
2021-09-01 03:00:00,4.7878e6,0.0,15.942,1,244,3,9,3,2021,"""no_holiday"""
2021-09-01 04:00:00,4.7038e6,0.0,15.792001,1,244,3,9,4,2021,"""no_holiday"""


# Cyclical Encoding

In [136]:
def sin_transformer(period):
    return FunctionTransformer(lambda x: np.sin(x / period * 2 * np.pi))


def cos_transformer(period):
    return FunctionTransformer(lambda x: np.cos(x / period * 2 * np.pi))


def encode_cyclically(column_name, periodicity, table):
    # Create sin and cos encoding
    table = table.with_columns(
        sin_transformer(periodicity)
        .fit_transform(table[column_name])
        .alias(f"{column_name}_sin")
    )

    table = table.with_columns(
        cos_transformer(periodicity)
        .fit_transform(table[column_name])
        .alias(f"{column_name}_cos")
    )
    # Drop the old column
    table = table.drop(column_name)

    return table

In [137]:
# Dictionary with column name and calendar periodicity
calendar_features = {
    "day_of_month": 31,
    "day_of_year": 365,
    "day_of_week": 7,
    "month": 12,
    "hour": 24,
}

for column_name, periodicity in calendar_features.items():
    df = encode_cyclically(column_name, periodicity, df)

# sklearn Pipeline

In [138]:
cat_cols = ["holiday_name"]

num_cols = df.select(
    cs.contains(
        "soil_temperature_7_to_28cm",
        "shortwave_radiation",
    )
).columns + ["year"]

manual_cols = df.select(pl.selectors.contains("_cos", "_sin", "is_")).columns

In [139]:
df.drop(manual_cols + cat_cols + num_cols)

datetime,kWh
datetime[μs],f64
2021-09-01 00:00:00,5.1633e6
2021-09-01 01:00:00,5.0777e6
2021-09-01 02:00:00,4.9315e6
2021-09-01 03:00:00,4.7878e6
2021-09-01 04:00:00,4.7038e6
…,…
2024-08-31 19:00:00,5.3614e6
2024-08-31 20:00:00,5.3775e6
2024-08-31 21:00:00,5.2130e6


In [140]:
numeric_transformer = Pipeline(
    steps=[("imputer", SimpleImputer(strategy="mean")), ("scaler", StandardScaler())]
)

In [141]:
categorical_transformer = Pipeline(
    steps=[
        (
            "encoder",
            OneHotEncoder(sparse_output=False, handle_unknown="ignore"),
        ),
    ]
)

In [142]:
column_transformer = ColumnTransformer(
    transformers=[
        ("numeric", numeric_transformer, num_cols),
        ("categorical", categorical_transformer, cat_cols),
    ],
    remainder="passthrough",
)

In [143]:
preprocessor = Pipeline(
    steps=[
        ("column_transformer", column_transformer),
        (
            "variance_threshold",
            VarianceThreshold(threshold=0.0),
        ),  # Drops constant columns after transformations
    ]
)

# Setup of Wide Data Format

- Want 24 hour chunks
    - $y \in R^{(24,)}$
    - $X \in R^{(24*d,)}$

Start by preprocessing the data in hourly frequency:

In [144]:
df_train = df.filter(
    (pl.col("datetime") >= pl.datetime(2021, 9, 1))
    & (pl.col("datetime") <= pl.datetime(2022, 8, 31))
).to_pandas()

df_val = df.filter(
    (pl.col("datetime") >= pl.datetime(2022, 9, 1))
    & (pl.col("datetime") <= pl.datetime(2023, 8, 31))
).to_pandas()

In [145]:
X_train = df_train.drop(columns=["datetime", "kWh"])
X_val = df_val.drop(columns=["datetime", "kWh"])

y_train = df_train["kWh"]
y_val = df_val["kWh"]

In [146]:
fitted_preprocessor = preprocessor.fit(X_train)

X_train_preprocessed = pd.DataFrame(
    fitted_preprocessor.transform(X_train),
    columns=fitted_preprocessor.get_feature_names_out(),
)

X_val_preprocessed = pd.DataFrame(
    fitted_preprocessor.transform(X_val),
    columns=fitted_preprocessor.get_feature_names_out(),
)

In [147]:
df_train_preprocessed = pd.concat(
    [df_train.filter(["datetime", "kWh"]), X_train_preprocessed], axis=1
)

df_val_preprocessed = pd.concat(
    [df_val.filter(["datetime", "kWh"]), X_val_preprocessed], axis=1
)

Then get the datasets into wide format separately:

- by X and y
- by training and validation

In [148]:
def aggregate_to_daily(tbl):

    out_df = (
        pl.from_pandas(tbl)
        # .drop("kWh")
        .with_columns(
            date=pl.col("datetime").dt.date(),
            hour=pl.col("datetime").dt.hour().cast(pl.Utf8).str.zfill(2),
        )
        .drop("datetime")
        .unpivot(index=["date", "hour"])
        .with_columns(
            variable=pl.concat_str(
                [pl.col("variable"), pl.col("hour")], separator="_hour"
            )
        )
        .sort(["date", "hour", "variable"], descending=[False, False, True])
        .drop("hour")
        .pivot(index="date", values="value", on="variable")
        .drop_nulls()
        .to_pandas()
    )

    return out_df

In [149]:
df_train_preprocessed_aggregated = aggregate_to_daily(df_train_preprocessed)
df_val_preprocessed_aggregated = aggregate_to_daily(df_val_preprocessed)

df_train_preprocessed_aggregated.head()

,date,remainder__month_sin_hour00,remainder__month_cos_hour00,remainder__hour_sin_hour00,remainder__hour_cos_hour00,remainder__day_of_year_sin_hour00,remainder__day_of_year_cos_hour00,remainder__day_of_week_sin_hour00,remainder__day_of_week_cos_hour00,remainder__day_of_month_sin_hour00,remainder__day_of_month_cos_hour00,numeric__year_hour00,numeric__Zurich_soil_temperature_7_to_28cm_hour00,numeric__Zurich_shortwave_radiation_hour00,kWh_hour00,categorical__holiday_name_no_holiday_hour00,categorical__holiday_name_Weihnachten_hour00,categorical__holiday_name_Tag der Arbeit_hour00,categorical__holiday_name_Stephanstag_hour00,categorical__holiday_name_Pfingstmontag_hour00,categorical__holiday_name_Ostermontag_hour00,categorical__holiday_name_Neujahrestag_hour00,categorical__holiday_name_Nationalfeiertag_hour00,categorical__holiday_name_Karfreitag_hour00,categorical__holiday_name_Berchtoldstag_hour00,categorical__holiday_name_Auffahrt_hour00,remainder__month_sin_hour01,remainder__month_cos_hour01,remainder__hour_sin_hour01,remainder__hour_cos_hour01,remainder__day_of_year_sin_hour01,remainder__day_of_year_cos_hour01,remainder__day_of_week_sin_hour01,remainder__day_of_week_cos_hour01,remainder__day_of_month_sin_hour01,remainder__day_of_month_cos_hour01,numeric__year_hour01,numeric__Zurich_soil_temperature_7_to_28cm_hour01,numeric__Zurich_shortwave_radiation_hour01,kWh_hour01,categorical__holiday_name_no_holiday_hour01,categorical__holiday_name_Weihnachten_hour01,categorical__holiday_name_Tag der Arbeit_hour01,categorical__holiday_name_Stephanstag_hour01,categorical__holiday_name_Pfingstmontag_hour01,categorical__holiday_name_Ostermontag_hour01,categorical__holiday_name_Neujahrestag_hour01,categorical__holiday_name_Nationalfeiertag_hour01,categorical__holiday_name_Karfreitag_hour01,categorical__holiday_name_Berchtoldstag_hour01,categorical__holiday_name_Auffahrt_hour01,remainder__month_sin_hour02,remainder__month_cos_hour02,remainder__hour_sin_hour02,remainder__hour_cos_hour02,remainder__day_of_year_sin_hour02,remainder__day_of_year_cos_hour02,remainder__day_of_week_sin_hour02,remainder__day_of_week_cos_hour02,remainder__day_of_month_sin_hour02,remainder__day_of_month_cos_hour02,numeric__year_hour02,numeric__Zurich_soil_temperature_7_to_28cm_hour02,numeric__Zurich_shortwave_radiation_hour02,kWh_hour02,categorical__holiday_name_no_holiday_hour02,categorical__holiday_name_Weihnachten_hour02,categorical__holiday_name_Tag der Arbeit_hour02,categorical__holiday_name_Stephanstag_hour02,categorical__holiday_name_Pfingstmontag_hour02,categorical__holiday_name_Ostermontag_hour02,categorical__holiday_name_Neujahrestag_hour02,categorical__holiday_name_Nationalfeiertag_hour02,categorical__holiday_name_Karfreitag_hour02,categorical__holiday_name_Berchtoldstag_hour02,categorical__holiday_name_Auffahrt_hour02,remainder__month_sin_hour03,remainder__month_cos_hour03,remainder__hour_sin_hour03,remainder__hour_cos_hour03,remainder__day_of_year_sin_hour03,remainder__day_of_year_cos_hour03,remainder__day_of_week_sin_hour03,remainder__day_of_week_cos_hour03,remainder__day_of_month_sin_hour03,remainder__day_of_month_cos_hour03,numeric__year_hour03,numeric__Zurich_soil_temperature_7_to_28cm_hour03,numeric__Zurich_shortwave_radiation_hour03,kWh_hour03,categorical__holiday_name_no_holiday_hour03,categorical__holiday_name_Weihnachten_hour03,categorical__holiday_name_Tag der Arbeit_hour03,categorical__holiday_name_Stephanstag_hour03,categorical__holiday_name_Pfingstmontag_hour03,categorical__holiday_name_Ostermontag_hour03,categorical__holiday_name_Neujahrestag_hour03,categorical__holiday_name_Nationalfeiertag_hour03,categorical__holiday_name_Karfreitag_hour03,categorical__holiday_name_Berchtoldstag_hour03,categorical__holiday_name_Auffahrt_hour03,remainder__month_sin_hour04,remainder__month_cos_hour04,remainder__hour_sin_hour04,remainder__hour_cos_hour04,remainder__day_of_year_sin_hour04,remainder__day_of_year_cos_hour04,remainder__day_of_week_sin_hour04,remain

In [150]:
df_train_preprocessed_aggregated.shape

(364, 601)

In [151]:
df_val_preprocessed_aggregated.shape

(364, 601)

In [152]:
(df_train_preprocessed.shape[1] - 1) * 24

600

# Hyperparameter Tuning

## Manual test

In [169]:
X_train_preprocessed_aggregated = df_train_preprocessed_aggregated.loc[
    :, ~df_train_preprocessed_aggregated.columns.str.contains("kWh_|date")
].to_numpy()

X_val_preprocessed_aggregated = df_val_preprocessed_aggregated.loc[
    :, ~df_train_preprocessed_aggregated.columns.str.contains("kWh_|date")
].to_numpy()

In [162]:
y_train_aggregated = df_train_preprocessed_aggregated.loc[
    :, df_train_preprocessed_aggregated.columns.str.contains("kWh_")
].to_numpy()

y_val_aggregated = df_val_preprocessed_aggregated.loc[
    :, df_train_preprocessed_aggregated.columns.str.contains("kWh_")
].to_numpy()

In [260]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping


def rmse(y_true, y_pred):
    return tf.sqrt(tf.reduce_mean(tf.square(y_true - y_pred)))


# Define the model
model = Sequential(
    [
        Input(shape=(X_train_preprocessed_aggregated.shape[1],)),
        Dense(64, activation="relu"),
        Dense(64, activation="relu"),
        Dense(64, activation="relu"),
        Dense(24),
    ]
)

# Compile the model
model.compile(
    optimizer=Adam(learning_rate=0.001), loss=rmse, metrics=[RootMeanSquaredError()]
)

# Set up Tensorboard
log_dir = "logs/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

# Set up callbacks
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)
early_stopping = EarlyStopping(
    monitor="val_loss", patience=10, restore_best_weights=True
)

# Train the model
history = model.fit(
    X_train_preprocessed_aggregated,
    y_train_aggregated,
    validation_data=(X_val_preprocessed_aggregated, y_val_aggregated),
    epochs=100,
    batch_size=32,
    callbacks=[tensorboard_callback, early_stopping],
    verbose=1,
)

Epoch 1/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - loss: 6586298.0000 - root_mean_squared_error: 6588279.5000 - val_loss: 6195940.0000 - val_root_mean_squared_error: 6237451.0000
Epoch 2/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 6620260.0000 - root_mean_squared_error: 6621357.0000 - val_loss: 6195933.0000 - val_root_mean_squared_error: 6237445.5000
Epoch 3/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 6582893.5000 - root_mean_squared_error: 6584730.5000 - val_loss: 6195910.5000 - val_root_mean_squared_error: 6237423.0000
Epoch 4/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 6585698.0000 - root_mean_squared_error: 6589393.5000 - val_loss: 6195840.5000 - val_root_mean_squared_error: 6237353.5000
Epoch 5/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 6597641.0000 - root_mean_squared_error: 6600602.5000 - val_loss: 6195651.5000 - val_root_mean_squared_error: 6237166.0000
Epoch 6/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 6553337.5000 - root_mean_squared

In [261]:
y_preds_val = pd.DataFrame(
    model.predict(X_val_preprocessed_aggregated),
    columns=df_val_preprocessed_aggregated.filter(regex="kWh_").columns,
).assign(date=df_val_preprocessed_aggregated["date"])

y_preds_val.head()

12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 


,kWh_hour00,kWh_hour01,kWh_hour02,kWh_hour03,kWh_hour04,kWh_hour05,kWh_hour06,kWh_hour07,kWh_hour08,kWh_hour09,kWh_hour10,kWh_hour11,kWh_hour12,kWh_hour13,kWh_hour14,kWh_hour15,kWh_hour16,kWh_hour17,kWh_hour18,kWh_hour19,kWh_hour20,kWh_hour21,kWh_hour22,kWh_hour23,date
0,4987045.5,4855571.5,4781049.5,4698791.5,4627189.00,4891717.5,5410010.0,5875720.5,6162613.5,6230060.5,6273616.5,6309108.0,6112287.0,6027305.0,5908232.5,5847825.5,5889464.5,6039646.0,6186244.0,6084631.0,5849068.5,5592617.0,5458758.5,5202365.0,2022-09-01
1,5015276.0,4883063.0,4808113.5,4725391.5,4653381.50,4919411.5,5440638.0,5908984.0,6197501.5,6265328.5,6309130.0,6344822.0,6146889.5,6061427.0,5941683.5,5880927.5,5922807.5,6073839.0,6221262.0,6119073.5,5882184.0,5624276.5,5489659.5,5231814.0,2022-09-02
2,4394970.0,4279120.0,4213436.0,4140948.5,4077836.75,4310962.5,4767729.0,5178145.0,5430977.5,5490419.5,5528796.0,5560083.5,5386627.5,5311731.5,5206797.0,5153550.5,5190257.5,5322618.0,5451795.0,5362252.5,5154652.5,4928645.0,4810687.0,4584733.0,2022-09-03
3,4487458.0,4369167.0,4302102.0,4228089.5,4163649.50,4401681.5,4868058.5,5287115.0,5545265.0,5605956.5,5645144.0,5677086.0,5499980.5,5423511.5,5316365.5,5261999.5,5299481.5,5434624.5,5566523.5,5475095.0,5263125.5,5032363.5,4911918.5,4681211.5,2022-09-04
4,4886224.0,4757413.0,4684391.0,4603803.0,4533641.50,4792824.5,5300642.5,5756938.0,6038028.5,6104106.5,6146785.0,6181558.0,5988714.5,5905456.0,5788787.5,5729593.5,5770404.5,5917548.5,6061179.5,5961620.5,5730823.5,5479554.5,5348396.5,5097189.5,2022-09-05


In [262]:
val_preds = (
    y_preds_val.melt(id_vars="date", value_name="pred")
    .assign(
        hour=lambda x: x["variable"].str.extract(r"hour(\d{2})")[0].astype(int),
        datetime=lambda x: pd.to_datetime(x["date"])
        + pd.to_timedelta(x["hour"], unit="h"),
    )
    .sort_values("datetime")
    .filter(["datetime", "pred"])
    .merge(df_val[["datetime", "kWh"]], how="left", on="datetime")
)

val_preds.head()

,datetime,pred,kWh
0,2022-09-01 00:00:00,4987045.5,4.993638e+06
1,2022-09-01 01:00:00,4855571.5,4.863520e+06
2,2022-09-01 02:00:00,4781049.5,4.803350e+06
3,2022-09-01 03:00:00,4698791.5,4.626046e+06
4,2022-09-01 04:00:00,4627189.0,4.572908e+06


In [263]:
# Create the figure with step lines for both actual and predicted values
fig = px.line(
    val_preds,
    x="datetime",
    y=["kWh", "pred"],
    labels={
        "datetime": "Date",
        "value": "Energy Consumption (kWh)",
        "variable": "Series",
    },
    title="Actual vs Predicted Energy Consumption Over Time",
    line_shape="hv",  # Set line shape to horizontal-vertical for step chart
)

# Customize the layout
fig.update_layout(
    template="plotly_white",
    legend=dict(title=""),
    xaxis_title="Date",
    yaxis_title="Energy Consumption (kWh)",
)

# Show the figure
fig.show()

## RayTune Tuning Loop

In [29]:
def sarimax_trainable(config, eval_option="in-sample"):

    try:
        # Fit the model on the training data
        forecaster = ForecasterSarimax(
            regressor=Sarimax(
                order=(config["p"], config["d"], config["q"]),
                seasonal_order=(config["P"], config["D"], config["Q"], 168),
            )
        )

        forecaster.fit(
            y=df_train["kWh"],
            exog=df_train.drop(columns=["kWh"]),
            suppress_warnings=True,
        )

        if eval_option == "in-sample":
            # Get the Information Criterion
            inf_criterion = forecaster.regressor.get_info_criteria("aic")
            train.report({"loss": inf_criterion})
        elif eval_option == "out-of-sample":
            y_preds_val = validation(forecaster)
            loss = root_mean_squared_error(
                y_pred=y_preds_val["pred"], y_true=y_preds_val["kWh"]
            )
            train.report({"loss": loss})

    except Exception as e:
        tune.report(metric=float("inf"))

In [ ]:
# Need this line for locally defined modules to work with ray
# ray.init(runtime_env={"working_dir": "."}, ignore_reinit_error=True)
np.random.seed(42)

analysis = tune.run(
    tune.with_parameters(
        sarimax_trainable,
        eval_option="out-of-sample",
    ),
    config={
        "p": tune.randint(1, 168),
        "q": tune.randint(1, 168),
        "P": tune.randint(1, 24),
        "Q": tune.randint(1, 24),
        "d": tune.randint(1, 5),
        "D": tune.randint(1, 5),
    },
    metric="loss",
    mode="min",
    name="SARIMAX",
    search_alg=OptunaSearch(),
    time_budget_s=60 * 60 * 1.5,
    num_samples=-1,
    raise_on_failed_trial=False,
    trial_dirname_creator=lambda trial: f"{trial.trainable_name}_{trial.trial_id}",
)

c:\Users\mathi\miniconda3\envs\lapro\lib\site-packages\ray\_private\node.py:1362: ResourceWarning:

unclosed file <_io.TextIOWrapper name='C:\\Users\\mathi\\AppData\\Local\\Temp\\ray\\session_2024-10-30_09-59-24_723195_17952\\logs\\gcs_server.out' mode='a' encoding='utf-8'>

c:\Users\mathi\miniconda3\envs\lapro\lib\site-packages\ray\_private\node.py:1362: ResourceWarning:

unclosed file <_io.TextIOWrapper name='C:\\Users\\mathi\\AppData\\Local\\Temp\\ray\\session_2024-10-30_09-59-24_723195_17952\\logs\\gcs_server.err' mode='a' encoding='utf-8'>

c:\Users\mathi\miniconda3\envs\lapro\lib\site-packages\ray\_private\node.py:1367: ResourceWarning:

unclosed file <_io.TextIOWrapper name='C:\\Users\\mathi\\AppData\\Local\\Temp\\ray\\session_2024-10-30_09-59-24_723195_17952\\logs\\monitor.out' mode='a' encoding='utf-8'>

c:\Users\mathi\miniconda3\envs\lapro\lib\site-packages\ray\_private\node.py:1367: ResourceWarning:

unclosed file <_io.TextIOWrapper name='C:\\Users\\mathi\\AppData\\Local\\Te

2024-10-30 10:10:14,318	WARNING tune.py:219 -- Stop signal received (e.g. via SIGINT/Ctrl+C), ending Ray Tune run. This will try to checkpoint the experiment state one last time. Press CTRL+C (or send SIGINT/SIGKILL/SIGTERM) to skip. 
2024-10-30 10:10:14,421	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to 'C:/Users/mathi/ray_results/SARIMAX' in 0.1020s.


In [15]:
analysis.dataframe().to_csv("2_SARIMAX_trials.csv", index=False)

NameError: name 'analysis' is not defined

# Model Evaluation